In [ ]:
!pip install ipython-autotime
%load_ext autotime
!pip install transformers
!pip install -U accelerate
!pip install -U datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.1 MB/s eta 0:00:00
time: 28.4 s (started: 2023-12-16 06:27:29 +00:00)


In [ ]:
import pandas as pd
import numpy as np
import torch
import json
import copy
import re
from transformers import pipeline

time: 742 µs (started: 2023-12-16 10:19:20 +00:00)


In [250]:
from google.colab import drive
drive.mount('/content/drive')
train_spider_path = '/content/drive/MyDrive/spider/train_spider.json'
train_others_path = '/content/drive/MyDrive/spider/train_others.json'
spider_tables_json_path = '/content/drive/MyDrive/spider/tables.json'
spider_inferred_type_tags_path = '/content/drive/MyDrive/CS559_shared/implementation/5_spider_inferred_type_tags.txt'
spider_pos_tags_path = '/content/drive/MyDrive/CS559_shared/implementation/2_spider_question_pos_tags.txt'
q_start_index = 8000
q_end_index = 8659

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
time: 2.25 s (started: 2023-12-16 13:17:41 +00:00)


In [265]:
def get_db_id_list_for_all_question(train_spider_path, train_others_path):
  train_spider_json_file = open(train_spider_path)
  train_spider_list = json.load(train_spider_json_file)
  train_others_json_file = open(train_others_path)
  train_others_list = json.load(train_others_json_file)

  train_all = train_spider_list + train_others_list

  spider_question_db_ids = []
  for question_dict in train_all:
    spider_question_db_ids.append(question_dict['db_id'])

  return spider_question_db_ids

time: 537 µs (started: 2023-12-16 13:18:23 +00:00)


In [266]:
spider_question_db_ids = get_db_id_list_for_all_question(train_spider_path, train_others_path)
spider_question_db_ids = spider_question_db_ids[q_start_index: q_end_index]

print(spider_question_db_ids)
print(len(spider_question_db_ids))

['scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar', 'scholar'

In [267]:
def retrieve_data_and_tag(path):
    file = open(path, 'r')
    sentences, sentence_tags = [], []
    for line in file:
        # line = re.sub(r"\".*\"","\"xxx\"",line)
        line = re.sub(r"'", "", line)
        word_tag_list = line.split()
        words = []
        tags = []
        for w in word_tag_list:
            item = w.split('/')
            if (len(item[0]) == 0):
                continue

            words.append(item[0])
            if (len(item) < 2):
                print(path, item)
            tags.append(item[1])
        sentences.append(words)
        sentence_tags.append(tags)
        if (len(sentences) != len(sentence_tags)):
            print(line)
    return sentences, sentence_tags

time: 1.02 ms (started: 2023-12-16 13:18:26 +00:00)


In [268]:
spider_questions_tokens_list, spider_questions_type_tags_list = retrieve_data_and_tag(spider_inferred_type_tags_path)
spider_questions_tokens_list = spider_questions_tokens_list[q_start_index : q_end_index]
spider_questions_type_tags_list = spider_questions_type_tags_list[q_start_index : q_end_index]

print(spider_questions_tokens_list[0])
print(spider_questions_type_tags_list[0])
print('len spider_questions_tokens_list: ', spider_questions_tokens_list)
print('len spider_questions_type_tags_list: ', spider_questions_type_tags_list)
print(len(spider_questions_type_tags_list))

['how', 'many', 'papers', 'are', 'published', 'in', 'sigir', '?']
['O', 'O', 'TABLE', 'O', 'TABLEREF', 'COND', 'VALUE', 'O']
len spider_questions_tokens_list:  [['how', 'many', 'papers', 'are', 'published', 'in', 'sigir', '?'], ['number', 'of', 'papers', 'in', 'sigir', 'conference'], ['what', 'papers', 'have', 'fewer', 'than', '5', 'citations', 'by', 'ACL', 'papers', '?'], ['ACL', 'papers', 'with', 'less', 'than', '5', 'citations'], ['what', 'ACL', 'papers', 'have', 'less', 'than', '5', 'citations', '?'], ['what', 'papers', 'have', 'fewer', 'than', '5', 'citations', 'by', 'ACL', 'papers'], ['What', 'are', 'the', '5', 'most', 'recent', 'papers', 'of', 'Mirella', 'Lapata', '?'], ['What', 'were', 'the', 'papers', 'published', 'at', 'pldi', '2015', '?'], ['Papers', 'from', 'pldi', '2015', 'conference'], ['papers', 'published', 'at', 'pldi', '2015'], ['What', 'are', 'the', 'papers', 'from', 'pldi', '2015', '?'], ['Papers', 'from', 'pldi', '2015', '?'], ['Which', 'papers', 'were', 'accepted'

In [269]:
spider_questions_tokens_list, spider_questions_pos_tags_list = retrieve_data_and_tag(spider_pos_tags_path)
spider_questions_tokens_list = spider_questions_tokens_list[q_start_index : q_end_index]
spider_questions_pos_tags_list = spider_questions_pos_tags_list[q_start_index : q_end_index]

print(spider_questions_tokens_list[0])
print(spider_questions_pos_tags_list[0])
print('len spider_questions_tokens_list: ', spider_questions_tokens_list)
print('len spider_questions_pos_tags_list: ', spider_questions_pos_tags_list)
print(len(spider_questions_pos_tags_list))

['how', 'many', 'papers', 'are', 'published', 'in', 'sigir', '?']
['WRB', 'JJ', 'NNS', 'VBP', 'VBN', 'IN', 'NN', '.']
len spider_questions_tokens_list:  [['how', 'many', 'papers', 'are', 'published', 'in', 'sigir', '?'], ['number', 'of', 'papers', 'in', 'sigir', 'conference'], ['what', 'papers', 'have', 'fewer', 'than', '5', 'citations', 'by', 'ACL', 'papers', '?'], ['ACL', 'papers', 'with', 'less', 'than', '5', 'citations'], ['what', 'ACL', 'papers', 'have', 'less', 'than', '5', 'citations', '?'], ['what', 'papers', 'have', 'fewer', 'than', '5', 'citations', 'by', 'ACL', 'papers'], ['What', 'are', 'the', '5', 'most', 'recent', 'papers', 'of', 'Mirella', 'Lapata', '?'], ['What', 'were', 'the', 'papers', 'published', 'at', 'pldi', '2015', '?'], ['Papers', 'from', 'pldi', '2015', 'conference'], ['papers', 'published', 'at', 'pldi', '2015'], ['What', 'are', 'the', 'papers', 'from', 'pldi', '2015', '?'], ['Papers', 'from', 'pldi', '2015', '?'], ['Which', 'papers', 'were', 'accepted', 'in',

In [270]:
def get_db_tables_and_columns(spider_tables_info_dict, db_id):
  db_table_columns_dict = {}
  table_names = spider_tables_info_dict[db_id]['table_names_original']
  for table_index in range(len(table_names)):
    table_name = table_names[table_index]
    db_table_columns_dict[table_name] = [col_item[1] for col_item in spider_tables_info_dict[db_id]['column_names_original'] if col_item[0] == table_index ]

  # table_names are keys and table columns are values as list
  return db_table_columns_dict

time: 644 µs (started: 2023-12-16 13:18:28 +00:00)


In [271]:
def create_serialized_input(spider_questions_tokens_list, spider_questions_type_tags_list, spider_questions_pos_tags_list, spider_question_db_ids, spider_tables_json_path):
  spider_table_json_file = open(spider_tables_json_path)
  spider_table_info_list = json.load(spider_table_json_file)

  # storing spider table and infos as a dict
  spider_tables_info_dict={}
  for db_dict in spider_table_info_list:
    if db_dict['db_id'] not in spider_tables_info_dict:
      spider_tables_info_dict[db_dict['db_id']] = db_dict

  serialized_inputs = []

  for ind, question_tokens in enumerate(spider_questions_tokens_list):
    serialized_single_input = copy.deepcopy(question_tokens)

    # adding correspoinding type tags
    serialized_single_input.append('[TTAG]') # adding special token [TTAG]
    for ttag in spider_questions_type_tags_list[ind]:
      if ttag != 'O' and ttag != 'COND':
        serialized_single_input.append('TABLE')
      else:
        serialized_single_input.append('O')

    serialized_single_input.append('[POS]') # adding special token [POS]
    serialized_single_input = serialized_single_input + spider_questions_pos_tags_list[ind]

    # adding table names to the input
    db_id = spider_question_db_ids[ind]
    db_tabels_columns_dict = get_db_tables_and_columns(spider_tables_info_dict, db_id)
    serialized_single_input.append('[T]')
    # obtainig related database table and column information
    for key, value in db_tabels_columns_dict.items():
      key = key.replace(' ', '_') # Remove spaces from table names
      serialized_single_input.append(key)

    serialized_inputs.append(serialized_single_input)

  return serialized_inputs

time: 1.17 ms (started: 2023-12-16 13:18:28 +00:00)


In [272]:
serialized_inputs_token_list = create_serialized_input(spider_questions_tokens_list, spider_questions_type_tags_list, spider_questions_pos_tags_list, spider_question_db_ids, spider_tables_json_path)
print(serialized_inputs_token_list[0])
print(len(serialized_inputs_token_list))


['how', 'many', 'papers', 'are', 'published', 'in', 'sigir', '?', '[TTAG]', 'O', 'O', 'TABLE', 'O', 'TABLE', 'O', 'TABLE', 'O', '[POS]', 'WRB', 'JJ', 'NNS', 'VBP', 'VBN', 'IN', 'NN', '.', '[T]', 'venue', 'author', 'dataset', 'journal', 'keyphrase', 'paper', 'cite', 'paperDataset', 'paperKeyphrase', 'writes']
659
time: 43.8 ms (started: 2023-12-16 13:18:28 +00:00)


In [273]:
spider_questions_tokens_list[0]

['how', 'many', 'papers', 'are', 'published', 'in', 'sigir', '?']

time: 3.67 ms (started: 2023-12-16 13:18:28 +00:00)


In [274]:
inputs = [" ".join(in_token_list) for in_token_list in serialized_inputs_token_list]
print('total input number: ', len(inputs))
print(inputs[:3])

total input number:  659
['how many papers are published in sigir ? [TTAG] O O TABLE O TABLE O TABLE O [POS] WRB JJ NNS VBP VBN IN NN . [T] venue author dataset journal keyphrase paper cite paperDataset paperKeyphrase writes', 'number of papers in sigir conference [TTAG] O O TABLE O TABLE TABLE [POS] NN IN NNS IN NN NN [T] venue author dataset journal keyphrase paper cite paperDataset paperKeyphrase writes', 'what papers have fewer than 5 citations by ACL papers ? [TTAG] O TABLE O O O O TABLE O TABLE TABLE O [POS] WDT NNS VBP JJR IN CD NNS IN NNP NNS . [T] venue author dataset journal keyphrase paper cite paperDataset paperKeyphrase writes']
time: 2.13 ms (started: 2023-12-16 13:18:28 +00:00)


In [275]:
# Load the model from the Hugging Face Hub
model_schema_tagger = pipeline(model="alpcaferoglu/t5-finetuned-dbtag-table-part")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


time: 2.85 s (started: 2023-12-16 13:18:28 +00:00)


In [276]:
# an example
example_index = 4 # 4, 9, 19 was problematic
example_input = inputs[example_index]
print(example_input)
example_nonserialized_input = spider_questions_tokens_list[example_index]
print('example_nonserialized_input: ', example_nonserialized_input)
print('len of example_nonserialized_input: ', len(example_nonserialized_input))

predicted_output = model_schema_tagger(example_input, min_new_tokens=len(example_nonserialized_input), max_new_tokens=len(example_nonserialized_input)+1)
generated_text = predicted_output[0]['generated_text']
print('generated_text: ', generated_text)
print('token len of generated_text: ', len(generated_text.split(" ")))

predicted_output = model_schema_tagger(example_input)
generated_text = predicted_output[0]['generated_text']
print('generated_text 2: ', generated_text)
print('token len of generated_text 2: ', len(generated_text.split(" ")))

what ACL papers have less than 5 citations ? [TTAG] O TABLE TABLE O O O O TABLE O [POS] WDT NNP NNS VBP JJR IN CD NNS . [T] venue author dataset journal keyphrase paper cite paperDataset paperKeyphrase writes
example_nonserialized_input:  ['what', 'ACL', 'papers', 'have', 'less', 'than', '5', 'citations', '?']
len of example_nonserialized_input:  9
generated_text:  O venue paper O O O O cite O
token len of generated_text:  9
generated_text 2:  O venue paper O O O O cite O O O O O O O O O O
token len of generated_text 2:  18
time: 2.79 s (started: 2023-12-16 13:18:31 +00:00)


Indexes with problems
* 4 --> employee is predicted as table but there is no such table
* 9
* 19

In [277]:
len(inputs)

659

time: 2.52 ms (started: 2023-12-16 13:18:34 +00:00)


In [278]:
generated_text_token_list = []
# for idx in range(len(inputs[:21])):
for idx in range(len(inputs)):
  input = inputs[idx]
  # print(input)
  nonserialized_input_tokens = spider_questions_tokens_list[idx]
  # print(nonserialized_input_tokens)
  # predicted_output = model_schema_tagger(input, min_length=len(nonserialized_input_tokens), max_length=len(nonserialized_input_tokens)+1)
  predicted_output = model_schema_tagger(input)
  generated_text_tokens = predicted_output[0]['generated_text'].split()
  generated_text_token_list.append(generated_text_tokens)



time: 15min 45s (started: 2023-12-16 13:18:34 +00:00)


In [279]:
# writing whole dataset into text file

path = '/content/drive/MyDrive/CS559_shared/implementation/spider_inferred_table_tags_' + str(q_start_index) + '-' + str(q_end_index) + '.txt'
with open(path, 'w') as file_handle:
  for input_tokens, generated_text_tokens in zip(spider_questions_tokens_list, generated_text_token_list):
    for ind in range(len(input_tokens)):
      if ind < len(generated_text_tokens):
        token_with_type = input_tokens[ind]+'/'+generated_text_tokens[ind]+ '/ '
      else:
        token_with_type = input_tokens[ind]+'/O/ '
      # print(token_with_type, end='')
      file_handle.write(token_with_type)
    file_handle.write('\n')
    # print()

time: 11.1 ms (started: 2023-12-16 13:34:19 +00:00)


In [280]:
len(generated_text_token_list)

659

time: 2.51 ms (started: 2023-12-16 13:34:19 +00:00)


time: 9.36 ms (started: 2023-12-16 13:34:19 +00:00)
